In [6]:
ACTIONS = ['UP', 'RIGHT', 'DOWN', 'LEFT', 'WAIT', 'BOMB']

DELTAS = [(0, -1), (1, 0), (0, 1), (-1, 0)]
for (dx, dy), action in zip(DELTAS, ACTIONS):
    print(dx, dy)
    print(action)

0 -1
UP
1 0
RIGHT
0 1
DOWN
-1 0
LEFT


In [3]:
print(test)



None
